## Проверка гепотез

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt

Подгрузим данные:

In [3]:
df_tracks = pd.read_csv('/content/df_tracks.csv')
df_tracks.head()

,Unnamed: 0,name,duration,explicit,track number,popularity,speechiness,tempo,dance-valence,feat,loud-energy
0,0,Born With A Beer In My Hand,188.0,0,1.0,77.0,0.0401,148.026,0.322848,0,-4.434750
1,1,Last Night,163.0,1,2.0,89.0,0.0389,203.759,0.235176,0,-3.682800
2,2,Everything I Love,187.0,0,3.0,80.0,0.0261,103.983,0.379424,0,-3.197482
3,3,Man Made A Bar (feat. Eric Church),191.0,0,4.0,81.0,0.0309,147.984,0.243522,1,-3.824584
4,4,Devil Don’t Know,205.0,0,5.0,76.0,0.0272,125.002,0.171190,0,-3.750272


In [4]:
df_tracks.drop('Unnamed: 0', axis=1, inplace=True)

Во время работы с данными у нас возникло несколько гипотез, которые мы хотели бы проверить:

### Гипотеза 1: ожидаемая популярность песен с матами равна ожидаемой популярности песен без них.  
### Альтернативная гипотеза: ожидаемая популярность песен с матами больше.
$$H_0: \mu_X = \mu_Y$$
$$H_A: \mu_X > \mu_Y$$
, где $\mu_X$ - ожидаемая популярность песен с матами, $\mu_Y$ - ожидаемая популярность песен без матов. \
Разделим данные по популярности:

In [5]:
X = df_tracks[(df_tracks['explicit'] == 1)]['popularity']
Y = df_tracks[(df_tracks['explicit'] == 0)]['popularity']

In [6]:
X.shape, Y.shape

((1439,), (1806,))

Данных у нас достаточно, чтобы говорить о применимости ЦПТ. Проверим гипотезу на уровне $5\%$ с помощью одностороннего $t$-теста (применяем его, так как не знаем настоящую теоритическую дисперсию):

In [7]:
sts.ttest_ind(X, Y, alternative='greater')

Ttest_indResult(statistic=11.445165423556311, pvalue=4.5977198463313635e-30)

Таким образом, гипотеза о равенстве отвергается в пользу гипотезы о том, что ожидаемая популярность песен с матами больше.

### Гипотеза 2: между номером трека в альбоме и популярностью нет зависимости
### Альтернативная гипотеза: между номером трека в альбоме и популярностью есть отрицательная зависимость
$$H_0: \rho = 0$$
$$H_A: \rho < 0$$
, где $\rho$ - коэффициент корреляции между номером трека в альбоме и популярностью. \
Посчитаем $\rho_{obs}$: 

In [8]:
rho_obs = np.corrcoef(df_tracks['popularity'], df_tracks['track number'])[0, 1]
rho_obs

-0.17111683971304037

Для проверки гипотезы проведем перестановочный тест:

In [9]:
corr_list = []
Y_const = df_tracks['popularity']
for i in range(10000):
    X_per = np.random.permutation(df_tracks['track number'])
    corr_coef = np.corrcoef(X_per, Y_const)[0, 1]
    corr_list.append(corr_coef)

Выведем критическое значение для уровня значимости $5\%$ $\rho_{crit}$:

In [10]:
lower_bound = np.percentile(corr_list, q=5)
lower_bound

-0.028712314474280198

Поскольку $\rho_{obs}$ < $\rho_{crit}$, нулевая гипотеза отвергается в пользу гипотезы о наличии отрицательной взаимосвязи между номером трека в альбоме и популярностью.

### Гипотеза 3: ожидаемая популярность песен с другими артистами равна ожидаемой популярности сольных песен
### Альтернативная гипотеза: ожидаемая популярность песен с другими артистами больше ожидаемой популярности сольных песен
$$H_0: \mu_X - \mu_Y= 0$$
$$H_A: \mu_X - \mu_Y> 0$$
,где $\mu_X$ - ожидаемая популярность песен с другими артистами, $\mu_Y$ - ожидаемая популярность сольных песен.

Разделим данные по популярности:

In [11]:
X = df_tracks[(df_tracks['feat'] == 1)]['popularity']
Y = df_tracks[(df_tracks['feat'] == 0)]['popularity']
X.shape, Y.shape

((312,), (2933,))

Найдем $\Delta_{obs}=\bar{X}-\bar{Y}$

In [12]:
X.mean() - Y.mean()

1.3745082483149318

Применим односторонний наивный бутстрэп, и проверим гипотезу на уровне $5\%$:

In [13]:
np.random.seed(15)
X_boot = np.random.choice(X, (10000, 312))
Y_boot = np.random.choice(Y, (10000, 2933))
X_mean = X_boot.mean(axis=1)
Y_mean = Y_boot.mean(axis=1)
deltha = X_mean-Y_mean

In [14]:
upper_bound = np.percentile(deltha, q=95)
upper_bound

2.8201236263736194

При данном уровне значимости нет оснований отвергать гипотезу о равенстве ожидаемых популярностей.